In [1]:
from bonito_pretrained import BonitoPretrained
checkpoint_path = 'checkpoints_pl/bonito_tl_0/epoch=0-step=4450000.ckpt'
model = BonitoPretrained(pretrained_lr=1e-4, my_layers_lr=1e-3, warmup_steps = 10000).load_from_checkpoint(checkpoint_path)

/home/jovyan/my-conda-envs/RNAmodEnv/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/jovyan/my-conda-envs/RNAmodEnv/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warn(f"Failed to load image Python extension: {e}")


In [12]:
from dataloading import get_my_dataset, get_kfold_split_func
from torch.utils.data import DataLoader

train_dset, valid_dset = get_my_dataset(valid_limit=0, window=1000, pos_files = 'pos_2022', neg_files='neg_2022', split_method=get_kfold_split_func(total_k=5, current_k=4))
valid_loader = DataLoader(valid_dset, batch_size=256, num_workers=32,
                          pin_memory=True, persistent_workers=True)

valid files indicies
[1, 11, 12, 13, 15, 16, 20, 23, 24, 26, 31, 39, 40, 44, 47, 52, 55, 57, 67, 71, 75, 86, 87, 88, 91, 96, 97, 101, 108, 111, 112, 114, 117, 119, 125, 137, 140, 141, 150, 154, 178, 186, 194, 195, 206, 207, 214, 216, 222, 229, 248, 253, 258, 266, 269]
[1, 4, 15, 20, 25, 29, 30, 31, 34, 35, 37, 41, 51, 61, 67, 79, 80, 84, 95, 97, 104, 109, 111, 120, 122, 126, 133, 135, 146, 151, 153, 160, 161, 170, 194, 199, 202, 205, 206, 214, 216, 218, 232, 234, 242, 244, 249, 257, 259, 260, 267, 271, 275, 277, 284, 288, 289, 291, 296, 298, 299, 304, 317, 318, 330, 336, 338, 342, 343, 345, 356, 366, 367, 374, 376, 382, 384, 401, 416, 426, 429, 435, 436, 441, 442, 461, 462, 464, 465, 470, 474, 476, 477, 482, 483, 490, 492, 497, 499, 500, 502, 510, 517, 519]
train files indicies
[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 14, 17, 18, 19, 21, 22, 25, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 41, 42, 43, 45, 46, 48, 49, 50, 51, 53, 54, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 72, 73, 74,

In [ ]:
# import pytorch_lightning as pl
# trainer= pl.Trainer(accelerator='gpu', benchmark=True, precision=16)
# # trainer.test(model, valid_loader)
# trainer.validate(model, valid_loader)

In [12]:
from skorch import NeuralNetClassifier
model_skorch = NeuralNetClassifier(model)
x,y = next(iter(valid_loader))


In [25]:
from cleanlab.classification import CleanLearning
data = x.numpy()
labels = y.numpy().flatten().astype(np.int64)
# labels
#TODO i have to put predictions into the un-sparse format (0,1 vs [1,0],[0,1])
# label_issues_info = CleanLearning(clf=model_skorch).find_label_issues(data,labels)

In [27]:
import numpy as np
def probs_to_bin(probs):
    res = []
    for pos_prob in probs:
        pos_prob = pos_prob[0]
        res.append([1-pos_prob, pos_prob])
    predz = np.array(res)
    return predz

In [28]:
from cleanlab.filter import find_label_issues
# from cleanlab.classification import CleanLearning
from torch import nn

x,y = next(iter(valid_loader))
probs = nn.Softmax(dim=0)(model(x)).detach().numpy()
probs = probs_to_bin(probs)

ordered_label_issues = find_label_issues(
    labels=y.numpy().flatten().astype(np.int32),
    pred_probs=probs,  # out-of-sample predicted probabilities from any model
    return_indices_ranked_by='self_confidence',
)

In [41]:
ordered_label_issues
print(len(ordered_label_issues))
y.numpy()[ordered_label_issues]


104


array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],